In [2]:
import re
from dataclasses import dataclass
from typing import List, Union

@dataclass
class Relationship:
    """Data class to store relationship information with type hints"""
    image_id: str
    subject: str
    subject_bbox: Union[List[int], str]  # can be list or 'n/a'
    relationship: str
    object: str
    object_bbox: Union[List[int], str]   # can be list or 'n/a'

def parse_line(line: str) -> Relationship:
    # Pattern matches either [x, y, z, w] or n/a
    bbox_pattern = r'\[[\d\s,]+\]|n/a'
    
    # Find all bbox values in the line
    bboxes = re.findall(bbox_pattern, line)
    if len(bboxes) != 2 and 'n/a' not in line:
        raise ValueError(f"Expected 2 bbox values, found {len(bboxes)}: {line}")
    
    # Replace bbox values with placeholders to safely split the remaining fields
    line_with_placeholders = re.sub(bbox_pattern, 'BBOX_PLACEHOLDER', line)
    fields = [f.strip() for f in line_with_placeholders.split(',')]
    
    if len(fields) != 6:
        raise ValueError(f"Expected 6 fields, found {len(fields)}: {line}")
    
    # Parse bbox values
    def parse_bbox(bbox: str) -> Union[List[int], str]:
        if bbox == 'n/a':
            return 'n/a'
        # Extract numbers from bracket notation
        numbers = re.findall(r'-?\d+', bbox)
        return [int(n) for n in numbers]
    
    # Replace placeholders with actual bbox values
    bbox_idx = 0
    fields_with_bboxes = []
    for field in fields:
        if field.strip() == 'BBOX_PLACEHOLDER':
            fields_with_bboxes.append(bboxes[bbox_idx])
            bbox_idx += 1
        else:
            fields_with_bboxes.append(field)
    
    return Relationship(
        image_id=fields_with_bboxes[0],
        subject=fields_with_bboxes[1],
        subject_bbox=parse_bbox(fields_with_bboxes[2]),
        relationship=fields_with_bboxes[3],
        object=fields_with_bboxes[4],
        object_bbox=parse_bbox(fields_with_bboxes[5])
    )

def parse_relationship_csv(file_path: str) -> List[Relationship]:
    """
    Parse relationship CSV file with proper handling of bracketed values
    
    Args:
        file_path: Path to the CSV file
    Returns:
        List of Relationship objects
    """
    relationships = []
    
    with open(file_path, 'r') as f:
        # Skip header
        next(f)
        
        for line_num, line in enumerate(f, start=2):  # start=2 to account for header
            try:
                relationship = parse_line(line.strip())
                relationships.append(relationship)
            except Exception as e:
                print(f"Error parsing line {line_num}: {line.strip()}")
                print(f"Error details: {str(e)}")
                continue
    
    return relationships

# Example usage
if __name__ == "__main__":
    # Sample data
    sample_data = """image_id,subject,subject_bbox,relationship,object,object_bbox
1,man,[150, 250, 60, 180],wearing,coat,[145, 240, 65, 100]
1,man,[150, 250, 60, 180],holding,bag,[160, 255, 40, 50]
1,woman,n/a,wearing,dress,[195, 260, 60, 110]
1,woman,[200, 270, 55, 170],nextTo,man,n/a
1,dog,[300, 320, 80, 90],hasAttribute,brown,n/a
1,dog,[300, 320, 80, 90],playingWith,stick,[310, 330, 20, 80]"""

    # Write sample data to a temporary file
    with open('temp.csv', 'w') as f:
        f.write(sample_data)

    # Parse the file
    relationships = parse_relationship_csv('temp.csv')
    
    # Print results
    for rel in relationships:
        print(rel)

Relationship(image_id='1', subject='man', subject_bbox=[150, 250, 60, 180], relationship='wearing', object='coat', object_bbox=[145, 240, 65, 100])
Relationship(image_id='1', subject='man', subject_bbox=[150, 250, 60, 180], relationship='holding', object='bag', object_bbox=[160, 255, 40, 50])
Relationship(image_id='1', subject='woman', subject_bbox='n/a', relationship='wearing', object='dress', object_bbox=[195, 260, 60, 110])
Relationship(image_id='1', subject='woman', subject_bbox=[200, 270, 55, 170], relationship='nextTo', object='man', object_bbox='n/a')
Relationship(image_id='1', subject='dog', subject_bbox=[300, 320, 80, 90], relationship='hasAttribute', object='brown', object_bbox='n/a')
Relationship(image_id='1', subject='dog', subject_bbox=[300, 320, 80, 90], relationship='playingWith', object='stick', object_bbox=[310, 330, 20, 80])
